# Введение в MapReduce модель на Python

In [1]:
from typing import Tuple, NamedTuple # requires python 3.6+
from typing import Iterator

**Модель элемента данных**

In [2]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [3]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

**Далее функции обработки данных**

In [4]:
def MAP(_, row: NamedTuple):
    if(row.gender == 'female'):
        yield (row.age, row)

def REDUCE(age: str, rows: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for u in rows:
        sum += u.social_contacts
        count += 1
    yield (age, sum / count)

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
    return [(user.id, user) for user in input_collection]

In [6]:
RECORDREADER()

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
    for it in nested_iterable:
        for elem in it:
            yield elem


def groupbykey(rows: Iterator):
    t = {}
    for (k, v) in rows:
        t[k] = t.get(k, []) + [v]
    return t.items()

In [8]:
res = list(flatten(map(lambda x: MAP(*x), RECORDREADER())))
res

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
res = list(groupbykey(res))
res

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [10]:
res = list(flatten(map(lambda x: REDUCE(*x), res)))
res

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [11]:
res = list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))
res

[(25, 370.0), (33, 800.0)]

## MapReduce

**Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных**.

**Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.**

In [12]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
      for element in iterable:
        yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
      t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




## Примеры

### SQL

In [13]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

### Matrix-Vector multiplication

In [14]:
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates: Tuple[int, int], value: int):
  i, j = coordinates
  yield (i, value * vec[j])

def REDUCE(line: int, values: Iterator[int]):
  sum = 0
  for v in values:
    sum +=  v
  yield (line, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i, j])

In [15]:
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.4684866245532278),
 (1, 2.4684866245532278),
 (2, 2.4684866245532278),
 (3, 2.4684866245532278),
 (4, 2.4684866245532278)]

### Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def MAP(docid: int, doc: str):
  for w in set(doc.split(' ')):
    yield (w, '{}'.format(docid))

def REDUCE(word: str, indexes: Iterator[int]):
  yield (word, sorted(indexes))

def RECORDREADER():
  for (docid, doc) in enumerate(documents):
    yield (docid, doc)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

### Word Count

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def MAP(docid: str, line: str):
  for w in line.split(' '):
    yield (w, 1)

def REDUCE(word: str, indexes: Iterator[int]):
  sum = 0
  for i in indexes:
    sum += i
  yield (word, sum)

def RECORDREADER():
  for (docid, doc) in enumerate(documents):
    for (lineid, line) in enumerate(doc.split('\n')):
      yield ('{}:{}'.format(docid, lineid), line)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

## MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
      for element in iterable:
        yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
      t[k2] = t.get(k2, []) + [v2]
    return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda row: MAP(*row), record_reader)), INPUTFORMAT())

    if COMBINER != None:
      map_partitions = map(lambda map_partition: flatten(map(lambda word: COMBINER(*word), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda word: REDUCE(*word), reduce_partition[1]))), reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(sum([len(v2) for (k2, v2) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
    return reduce_outputs


## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  def RECORDREADER(split):
    for (docid, doc) in enumerate(split):
      for (lineid, line) in enumerate(doc.split('\n')):
        yield ('{}:{}'.format(docid, lineid), line)
  split_size = (int)(len(documents) / maps)
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i : i + split_size])

def MAP(docid: str, line: str):
  for w in line.split(' '):
    yield (w, 1)

def REDUCE(word: str, indexes: Iterator[int]):
  sum = 0
  for i in indexes:
    sum += i
  yield (word, sum)


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(partitioned_output)

16 key-value pairs were sent over a network.
[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18)]), (1, [('is', 18), ('what', 10)])]


In [20]:
res = list(map(lambda record_reader: flatten(map(lambda row: MAP(*row), record_reader)), INPUTFORMAT()))
res

[<generator object flatten at 0x7e776ffbe650>,
 <generator object flatten at 0x7e776ffbe730>,
 <generator object flatten at 0x7e776ffbe810>]

In [21]:
res = list(res[0])
res

[('', 1),
 ('it', 1),
 ('is', 1),
 ('what', 1),
 ('it', 1),
 ('is', 1),
 ('it', 1),
 ('is', 1),
 ('what', 1),
 ('it', 1),
 ('is', 1),
 ('it', 1),
 ('is', 1),
 ('what', 1),
 ('it', 1),
 ('is', 1),
 ('', 1),
 ('what', 1),
 ('is', 1),
 ('it', 1),
 ('what', 1),
 ('is', 1),
 ('it', 1)]

In [22]:
list(list(INPUTFORMAT())[0])

[('0:0', ''),
 ('0:1', 'it is what it is'),
 ('0:2', 'it is what it is'),
 ('0:3', 'it is what it is'),
 ('1:0', ''),
 ('1:1', 'what is it'),
 ('1:2', 'what is it')]

In [23]:
list(INPUTFORMAT())

[<generator object INPUTFORMAT.<locals>.RECORDREADER at 0x7e776ffbec00>,
 <generator object INPUTFORMAT.<locals>.RECORDREADER at 0x7e776ffbec70>,
 <generator object INPUTFORMAT.<locals>.RECORDREADER at 0x7e776ffbece0>]

## Tera Sort

In [24]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  def RECORDREADER(split):
    for val in split:
      yield (val, None)
  split_size = (int)(input_values.shape[0] / maps)
  for i in range(0, input_values.shape[0], split_size):
    yield RECORDREADER(input_values[i : i + split_size])

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def MAP(val, _):
  yield (val, None)

def REDUCE(val, _):
  #yield (None, val)
   yield (val, None)


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(0.040262167752571076, None),
   (0.11099223932607039, None),
   (0.11811530570617712, None),
   (0.12708930847621136, None),
   (0.13926357008672874, None),
   (0.1470983392810753, None),
   (0.257680839510403, None),
   (0.2657172367982302, None),
   (0.28843480028225654, None),
   (0.3056797605636915, None),
   (0.38218790272868786, None),
   (0.40677657625799346, None)]),
 (1,
  [(0.538270416370419, None),
   (0.5554740967330576, None),
   (0.5635186281479725, None),
   (0.5658247625941342, None),
   (0.5721303329834616, None),
   (0.582249571119346, None),
   (0.5855701999224817, None),
   (0.6005556837215584, None),
   (0.6020352823047421, None),
   (0.7129201368121318, None),
   (0.7339116879502717, None),
   (0.7775886365624756, None),
   (0.8036530289533255, None),
   (0.8123970009694836, None),
   (0.8136211938401795, None),
   (0.9249792315764176, None),
   (0.9300119791549287, None),
   (0.9578719286427426, None)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [25]:
# Входной список чисел:
x = [1, 2, 3, 4, 5]
print(f'Входной список чисел: {x}')

def MAP(_, value):
  yield (1, value)

def REDUCE(_, values):
  max = values[0]
  for i in range(len(values)):
    if values[i] > max:
      max = values[i]
  yield max

def RECORDREADER():
  for (i, v) in enumerate(x):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)[-1]
print(f'Максимальное число: {output}')

Входной список чисел: [1, 2, 3, 4, 5]
Максимальное число: 5


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [26]:
# Входной список чисел:
x = [1, 2, 3, 4, 5]
print(f'Входной список чисел: {x}')

def MAP(_, value):
  yield (1, value)

def REDUCE(_, values):
  sum = 0
  count = 0
  for v in values:
    sum += v
    count += 1
  yield sum / count

def RECORDREADER():
  for (i, v) in enumerate(x):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)[-1]
print(f'Арифметическое среднее: {output}')

Входной список чисел: [1, 2, 3, 4, 5]
Арифметическое среднее: 3.0


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [27]:
# Входной список чисел:
x = [5, 4, 3, 2, 1]
print(f'Входной список чисел: {x}')

def MAP(_, value):
  yield (1, value)

def REDUCE(_, values):
  sum = 0
  count = 0
  for v in values:
    sum += v
    count += 1
  yield sum / count

def groupbykey_sorted(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = sorted(t.get(k2, []) + [v2])
  return t.items()


output = groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))
output = list(output)[-1]
print(f'Группировка по ключу без сортировки: {output}')

output = groupbykey_sorted(flatten(map(lambda x: MAP(*x), RECORDREADER())))
output = list(output)[-1]
print(f'Группировка по ключу с сортировкой: {output}')

Входной список чисел: [5, 4, 3, 2, 1]
Группировка по ключу без сортировки: (1, [5, 4, 3, 2, 1])
Группировка по ключу с сортировкой: (1, [1, 2, 3, 4, 5])


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [28]:
# Входной список чисел:
x = [1, 1, 2, 2, 3, 3, 4, 4, 5, 5]
print(f'Входной список чисел с дубликатами: {x}')

def MAP(_, value):
  yield (value, 1)

def REDUCE(key, _):
  yield key

def RECORDREADER():
  for (i, v) in enumerate(x):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f'Список уникальных чисел: {output}')

Входной список чисел с дубликатами: [1, 1, 2, 2, 3, 3, 4, 4, 5, 5]
Список уникальных чисел: [1, 2, 3, 4, 5]


In [29]:
d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid, lineid), line)

  split_size = int(np.ceil(len(documents) / maps))

  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i + split_size])

def MAP(docId: str, line: str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word: str, counts: Iterator[int]):
  yield word

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'a', 'banana', 'it']), (1, ['is', 'what'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [30]:
# Входной список пользователей:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(key, value):
  if value.gender == 'male':
    yield (value, value)

def REDUCE(*arg):
  yield arg

def RECORDREADER():
  for (i, v) in enumerate(input_collection):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = map(lambda x: x[0], output)
output = list(output)
print(f'Список пользователей мужского пола:')
print(output)

Список пользователей мужского пола:
[User(id=0, age=55, social_contacts=20, gender='male')]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [31]:
# Проекция на наших пользователей:
class UserFiltered(NamedTuple):
  id: int
  gender: str

# Входной список пользователей:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(key, value):
  user = UserFiltered(id=value.id, gender=value.gender)
  yield (user, user)

def REDUCE(key, values):
  yield (key, key)

def RECORDREADER():
  for (i, v) in enumerate(input_collection):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f'Вернем только id и пол человека:')
output

Вернем только id и пол человека:


[(UserFiltered(id=0, gender='male'), UserFiltered(id=0, gender='male')),
 (UserFiltered(id=1, gender='female'), UserFiltered(id=1, gender='female')),
 (UserFiltered(id=2, gender='female'), UserFiltered(id=2, gender='female')),
 (UserFiltered(id=3, gender='female'), UserFiltered(id=3, gender='female'))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [32]:
first_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

second_collection = [
    User(id=4, age=81, gender='male', social_contacts=228),
    User(id=5, age=150, gender='female', social_contacts=1),
]

def MAP(key, value):
  yield (value, value)

def REDUCE(key, values):
  yield (key, key)

def RECORDREADER():
  for (i, v) in enumerate(first_collection + second_collection):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f'Операция объединения:')
output

Операция объединения:


[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=4, age=81, social_contacts=228, gender='male'),
  User(id=4, age=81, social_contacts=228, gender='male')),
 (User(id=5, age=150, social_contacts=1, gender='female'),
  User(id=5, age=150, social_contacts=1, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [33]:
first_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

second_collection = [
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500)
]

def MAP(key, value):
  yield (value, value)

def REDUCE(key, values):
  if len(values) == 2:
    yield (key, key)

def RECORDREADER():
  for (i, v) in enumerate(first_collection + second_collection):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f'Операция пересечения:')
output

Операция пересечения:


[(User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [34]:
first_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

second_collection = [
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500)
]

def MAP(key, value):
  if value in first_collection:
    yield (value, 'R')
  else:
    yield (value, 'S')

def REDUCE(key, values):
  if values == ['R']:
    yield (key, key)

def RECORDREADER():
  for (i, v) in enumerate(first_collection + second_collection):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f'Операция разницы:')
output

Операция разницы:


[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [35]:
class User(NamedTuple):
  id: int
  userName: str
  age: str
  cityName: str

class UserFiltered(NamedTuple):
  id: int
  userName: str
  age: str

class City(NamedTuple):
  id: int
  cityName: str
  governor: str

class CityFiltered(NamedTuple):
  id: int
  cityName: str
  governor: str

class UserCity(NamedTuple):
  userId: int
  userName: str
  age: str
  city: str
  governor: str

first_collection = [
    User(id=0, userName='Maria', age='23', cityName='Samara'),
    User(id=1, userName='Dima', age='18', cityName='Moscow'),
    User(id=2, userName='Sasha', age='35', cityName='Tomsk'),
    User(id=3, userName='Ilya', age='22', cityName='Samara'),
]

second_collection = [
    City(id=0, cityName='Samara', governor='Azarov'),
    City(id=1, cityName='Moscow', governor='Vorobiev'),
    City(id=2, cityName='Tomsk', governor='Mazur'),
]

def MAP(key, value):
  if value in first_collection:
    yield (value.cityName, ('R', UserFiltered(id=value.id, userName=value.userName, age=value.age)))
  else:
    yield (value.cityName, ('S', CityFiltered(id=value.id, cityName=value.cityName, governor=value.governor)))

def REDUCE(key, values):
  list_R = [a for (x, a) in values if x == 'R'] # все возможные пользователи
  list_S = [c for (x, c) in values if x == 'S'] # все возможные города
  # И теперь каждый с каждым:
  for a in list_R: # пользователи
    for c in list_S: # города
      yield (key, UserCity(userId=a.id, userName=a.userName, age=a.age, city=c.cityName, governor=c.governor))

def RECORDREADER():
  for (i, v) in enumerate(first_collection + second_collection):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f'Операция естественного соединения:')
output

Операция естественного соединения:


[('Samara',
  UserCity(userId=0, userName='Maria', age='23', city='Samara', governor='Azarov')),
 ('Samara',
  UserCity(userId=3, userName='Ilya', age='22', city='Samara', governor='Azarov')),
 ('Moscow',
  UserCity(userId=1, userName='Dima', age='18', city='Moscow', governor='Vorobiev')),
 ('Tomsk',
  UserCity(userId=2, userName='Sasha', age='35', city='Tomsk', governor='Mazur'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [36]:
class User(NamedTuple):
  id: int
  age: str
  gender: str
  social_contacts: str

class UserFiltered(NamedTuple):
  age: str
  social_contacts: str

input_collection = [
    User(id=0, age=33, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(key, value):
  yield (value.age, value.social_contacts)

def REDUCE(key, values):
  sum = 0
  count = 0
  for v in values:
    sum += v
    count += 1
  yield (key, sum / count)

def RECORDREADER():
  for (i, v) in enumerate(input_collection):
    yield (i, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f'Операция группировки и агрегации:')
output

Операция группировки и агрегации:


[(33, 410.0), (25, 370.0)]

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [41]:
ROWS = 5
COLS = 4
STRIPES = 2

COLS_IN_STRIPE = COLS // STRIPES

M = [np.ones((ROWS, COLS_IN_STRIPE)) for i in range(STRIPES)]
v = [np.random.rand(COLS_IN_STRIPE) for i in range(STRIPES)]

def MAP(coordinates: Tuple[int, int], value: int):
  i, j = coordinates
  yield (i, value * vec[j])

def REDUCE(i: int, products: Iterator[NamedTuple]):
  print(i, products)
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  global vec
  for stripe_index, stripe in enumerate(M):
    vec = v[stripe_index]
    for i in range(stripe.shape[0]):
      for j in range(stripe.shape[1]):
        yield ((i, j), stripe[i, j])

print(M)
print(v)
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])]
[array([0.83044239, 0.2911758 ]), array([0.1154932 , 0.63715313])]
0 [0.8304423929529875, 0.29117579994127907, 0.11549320038179278, 0.6371531325307949]
1 [0.8304423929529875, 0.29117579994127907, 0.11549320038179278, 0.6371531325307949]
2 [0.8304423929529875, 0.29117579994127907, 0.11549320038179278, 0.6371531325307949]
3 [0.8304423929529875, 0.29117579994127907, 0.11549320038179278, 0.6371531325307949]
4 [0.8304423929529875, 0.29117579994127907, 0.11549320038179278, 0.6371531325307949]


[(0, 1.8742645258068542),
 (1, 1.8742645258068542),
 (2, 1.8742645258068542),
 (3, 1.8742645258068542),
 (4, 1.8742645258068542)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [37]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [42]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for j in range(J):
    for k in range(K):
      yield ((j, k), big_mat[j, k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(I):
    yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
  (i, k) = key
  yield ((i, k), sum(values))

# Проверьте своё решение

In [43]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [44]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [45]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for i in range(I):
    for j in range(J):
      yield ((0, i, j), small_mat[i, j])

  for j in range(J):
    for k in range(K):
      yield ((1, j, k), big_mat[j, k])

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1
  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))


def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)


def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_RECORDREADER(iterable):
  def func():
    for el in iterable:
      yield el
  return func


reference_solution = np.matmul(small_mat, big_mat)
partial_solution = MapReduce(RECORDREADER, MAP, REDUCE)
solution = MapReduce(GET_RECORDREADER(partial_solution), MAP_IDENTITY, REDUCE_AGREGATE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [46]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
  def RECORDREADER(index, matrix):
    rows, cols = matrix.shape
    for i in range(rows):
      for j in range(cols):
        yield ((index, i, j), matrix[i, j])

  yield RECORDREADER(0, small_mat)
  yield RECORDREADER(1, big_mat)

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1

  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
  def func():
    for el in iterable:
      yield el[1]
  return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
  for v in value[1]:
    solution_unpacked.append(v)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [47]:
I = 2
J = 3
K = 4
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
  def RECORDREADER(index, matrix, shift):
    rows, cols = matrix.shape
    for i in range(rows):
      for j in range(cols):
        yield ((index, i + shift, j), matrix[i, j])

  split_size = int(np.ceil(len(small_mat) / maps))

  for i in range(0, len(small_mat), split_size):
    yield RECORDREADER(0, small_mat[i:i + split_size], i)

  split_size = int(np.ceil(len(big_mat) / maps))

  for i in range(0, len(big_mat), split_size):
    yield RECORDREADER(1, big_mat[i:i + split_size], i)

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1

  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
  def func():
    for el in iterable:
      yield el[1]
  return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
  for v in value[1]:
    solution_unpacked.append(v)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true

18 key-value pairs were sent over a network.
24 key-value pairs were sent over a network.


True